Siin seadistame keskkonna ja loeme sisse toorandmed.

In [10]:
import pandas as pd
import json
import numpy as np

# Seadistame Pandase loetavuse huvides
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Laeme andmed
df = pd.read_csv('toorandmed_aasta.csv')

Samm 1: Abifunktsioonid JSON-i ja teksti puhastamiseks

Kuna paljud väljad on keerulistes JSON-listides, loome funktsiooni, mis sealt olulise info (nt keele nime või õppeastme) kätte saab.

In [11]:
def clean_json_field(x, key='et'):
    """Võtab JSON stringist välja konkreetse keelega väärtuse või koodi."""
    if pd.isna(x) or str(x).strip() in ["", "[]", "None"]:
        return np.nan
    try:
        # Puhastame CSV-st tulevad topeltjutumärgid
        cleaned_x = str(x).replace("''", "'").strip()
        data = json.loads(cleaned_x)

        if isinstance(data, list) and len(data) > 0:
            item = data[0]
            if isinstance(item, dict):
                # Eelistame eestikeelset nime, siis koodi
                return item.get(key) or item.get('code') or str(item)
        elif isinstance(data, dict):
            return data.get(key) or data.get('code')
        return str(data)
    except:
        # Kui JSON-ina laadimine ebaõnnestub, tagastame algse teksti puhastatuna
        return str(x).strip()

Samm 2: Veergude ühendamine ja eelistuste seadmine

Selles sammus kombineerime version ja general info. Kui versiooni all on info olemas, eelistame seda.

In [15]:
# 1. EELFILTRID
# Eemaldame ained, mis kestavad üle 1 semestri
df_filtered = df[df['additional_info__duration_in_semesters'] <= 1].copy()

# Eemaldame kaitsmise ja lõputöödega seotud ained
kaitsmise_keywords = ['kaitsmine', 'lõputöö', 'bakalaureusetöö', 'magistritöö', 'thesis', 'defence']
df_filtered = df_filtered[~df_filtered['title__et'].str.contains('|'.join(kaitsmise_keywords), case=False, na=False)]

# 2. TÄPSUSTATUD VEERGUDE VALIK (Vastavalt sinu faili päistele)
mappings = {
    'pealkiri': ('version__title__et', 'title__et'),
    'kirjeldus': ('version__overview__description__et', 'overview__description__et'),
    'õpiväljundid': ('version__overview__learning_outcomes_text_et', 'overview__learning_outcomes_text_et'),
    'eeldusained': ('additional_info__prerequisites', None), # Failis on see veerg olemas
    'ainepunktid': ('version__credits', 'credits'),
    'hindamine': ('additional_info__assessment_scale__et', None),
    'semester': ('version__target__semester__et', None),
    'õppeaste': ('version__additional_info__study_levels', None),
    'keel': ('version__target__language__et', 'general__input_languages'),
    'asukoht': ('version__target__course_main_structural_unit__city', None)
}

df_final = pd.DataFrame()
df_final['kood'] = df_filtered['code']

# Loome uued veerud vastavalt mappingule
for new_col, (v_col, g_col) in mappings.items():
    if v_col in df_filtered.columns:
        if g_col and g_col in df_filtered.columns:
            df_final[new_col] = df_filtered[v_col].fillna(df_filtered[g_col])
        else:
            df_final[new_col] = df_filtered[v_col]
    elif g_col and g_col in df_filtered.columns:
        df_final[new_col] = df_filtered[g_col]
    else:
        # Kui kumbagi veergu pole, tekitame tühja veeru, et hilisem kood ei puruneks
        df_final[new_col] = np.nan

# 3. JSON PUHASTUS JA HARD-FILTRID (Kontrollime veeru olemasolu enne puhastust)
keerulised_veerud = ['õppeaste', 'keel', 'eeldusained']

for col in keerulised_veerud:
    if col in df_final.columns:
        df_final[col] = df_final[col].apply(clean_json_field)

# Hard-filter: Eksam (Eristav hindamine)
if 'hindamine' in df_final.columns:
    df_final['on_eksam'] = df_final['hindamine'].str.contains('Eristav', na=False).map({True: 'Jah', False: 'Ei'})
else:
    df_final['on_eksam'] = "Teadmata"

print(f"Samm 2 valmis! Aineid alles: {len(df_final)}")

Samm 2 valmis! Aineid alles: 3098


Samm 3: Tunnuse "kirjeldus" loomine (RAG koondtekst)

Nüüd sisaldab koondtekst ka eeldusaineid ja õpiväljundeid, et RAG-süsteem saaks vastata keerukamatele küsimustele.

In [16]:
def loo_rag_tekst(row):
    # Koostame struktureeritud jutu aine kohta
    parts = [
        f"AINE: {row['pealkiri']} ({row['kood']})",
        f"MAHT: {row['ainepunktid']} EAP",
        f"HINDAMINE: {row['hindamine']} (Eksam: {row['on_eksam']})",
        f"EELDUSAINED: {row['eeldusained'] if pd.notna(row['eeldusained']) else 'Puuduvad'}",
        f"ÕPIVÄLJUNDID: {row['õpiväljundid']}",
        f"KIRJELDUS: {row['kirjeldus']}"
    ]
    return "\n".join([str(p) for p in parts if pd.notna(p) and str(p).strip() != ""])

df_final['kirjeldus_koond'] = df_final.apply(loo_rag_tekst, axis=1)

# Statistika tähemärkide kohta
print("--- Koondkirjelduse tähemärkide statistika ---")
print(df_final['kirjeldus_koond'].str.len().describe())

--- Koondkirjelduse tähemärkide statistika ---
count    3098.000000
mean      823.501937
std       462.319615
min       153.000000
25%       476.250000
50%       722.000000
75%      1058.750000
max      3982.000000
Name: kirjeldus_koond, dtype: float64


Samm 4: Kontroll ja kategooriad

See plokk vastab ülesande nõudele näidata enim levinud väärtusi.

In [17]:
print("\n--- PUUDUVATE VÄÄRTUSTE ARV ---")
print(df_final.isnull().sum())

for col in ['keel', 'semester', 'õppeaste', 'on_eksam']:
    print(f"\n--- TOP 5: {col} ---")
    print(df_final[col].value_counts().head(5))


--- PUUDUVATE VÄÄRTUSTE ARV ---
kood                  0
pealkiri              0
kirjeldus             0
õpiväljundid       1566
eeldusained        2537
ainepunktid           0
hindamine             0
semester             18
õppeaste             55
keel                  0
asukoht            1157
on_eksam              0
kirjeldus_koond       0
dtype: int64

--- TOP 5: keel ---
keel
eesti keel      2304
inglise keel     794
Name: count, dtype: int64

--- TOP 5: semester ---
semester
sügis    1546
kevad    1534
Name: count, dtype: int64

--- TOP 5: õppeaste ---
õppeaste
bakalaureuseõpe                               1842
magistriõpe                                    701
integreeritud bakalaureuse- ja magistriõpe     259
rakenduskõrgharidusõpe                         163
doktoriõpe                                      78
Name: count, dtype: int64

--- TOP 5: on_eksam ---
on_eksam
Jah    1810
Ei     1288
Name: count, dtype: int64


Samm 5: "Sissejuhatus andmeteadusesse" (Visualiseeritud väljund)

Vormistame selle aine info maksimaalselt loetavalt.

In [18]:
# Otsime aine (eeldame, et kood on LTAT.02.002 või pealkiri sarnane)
aine_mask = df_final['pealkiri'].str.contains("Sissejuhatus andmeteadusesse", na=False, case=False)

if any(aine_mask):
    aine = df_final[aine_mask].iloc[0]

    print("="*70)
    print(f"📘 AINEKAART: {aine['pealkiri']} ({aine['kood']})".center(70))
    print("="*70)

    col_layout = [
        ("Maht", f"{aine['ainepunktid']} EAP"),
        ("Õppeaste", aine['õppeaste']),
        ("Keel", aine['keel']),
        ("Semester", aine['semester']),
        ("Asukoht", aine['asukoht']),
        ("Hindamine", aine['hindamine']),
        ("Eksam", aine['on_eksam'])
    ]

    for label, val in col_layout:
        print(f"{label:<15}: {val}")

    print("-" * 70)
    print("🔗 EELDUSAINED:")
    print(aine['eeldusained'] if pd.notna(aine['eeldusained']) else "Puuduvad")

    print("\n🎯 ÕPIVÄLJUNDID:")
    print(aine['õpiväljundid'][:500] + "..." if len(str(aine['õpiväljundid'])) > 500 else aine['õpiväljundid'])

    print("\n📖 RAG KOONDTEKST (tunnus 'kirjeldus_koond'):")
    print(f"[Teksti pikkus: {len(aine['kirjeldus_koond'])} tähemärki]")
    print(f"{aine['kirjeldus_koond'][:300]}...") # Näitame algust
    print("="*70)
else:
    print("Ainet ei leitud.")

       📘 AINEKAART: Sissejuhatus andmeteadusesse (LTAT.02.002)        
Maht           : 6.0 EAP
Õppeaste       : bakalaureuseõpe
Keel           : inglise keel
Semester       : sügis
Asukoht        : nan
Hindamine      : Eristav (A, B, C, D, E, F, mi)
Eksam          : Jah
----------------------------------------------------------------------
🔗 EELDUSAINED:
Puuduvad

🎯 ÕPIVÄLJUNDID:
- Kursuse läbinud üliõpilane on võimeline:
- - Loetlema ning selgitama peamisi andmeteaduse alamvaldkondi, põhimõisteid ning põhiülesandeid
- - Teostama etteantud andmestikul kirjeldavat analüüsi
- - Valima ning teostama andmestikule sobivaid visualiseeringuid
- - Rakendama tavalisemaid klasterdusmeetodeid ning tõlgendama nende tulemusi
- - Valima ja rakendama lihtsamaid masinõppe algoritme, et luua ennustavaid mudeleid klassifikatsiooni ja regressiooni jaoks
- - Kasutama lihtsamaid hindamismeet...

📖 RAG KOONDTEKST (tunnus 'kirjeldus_koond'):
[Teksti pikkus: 1661 tähemärki]
AINE: Sissejuhatus andmeteadusesse